In [1]:
import geojson
from geopy.distance import geodesic

### This file cuts a GeoJSON at the closest point to a chosen lat/lng and discards the remainder, REPLACING the original file

### Use with caution

In [2]:
# Define these CAREFULLY!! - It will REPLACE the original file

################################
# path to geojson
file_path = '../docs/resources/geojson/train_mumbai_new_delhi.geojson'

# set coordinates to cut the trail / road / track
# Note: (latitude, longitude)
endpoint = (26.019811, 76.358374)
#################################

In [3]:
# support functions for cutting a hiking trail / road / train track

# find the closest point to the endpoint on the trail / road / track
def find_closest_point(feature, endpoint):
    closest_point = None
    min_distance = float("inf")
    for coord in feature["geometry"]["coordinates"]:
        # convert to (latitude, longitude) for geodesic calculation
        coord_latlon = (coord[1], coord[0])
        distance = geodesic(coord_latlon, endpoint).meters
        if distance < min_distance:
            min_distance = distance
            closest_point = coord
    return closest_point

#### Switch between the following two functions, to keep everything before or after the endpoint

In [4]:
# switch between these two functions to filter the trail,
# depending on whether you want to keep the trail before or after the endpoint

# filter the trail up to the closest point
def filter_trail(feature, closest_point):
    filtered_coords = []
    for coord in feature["geometry"]["coordinates"]:
        filtered_coords.append(coord)
        if coord == closest_point:  # stop at the closest point
            break
    return filtered_coords

# filter the trail starting from the closest point
# def filter_trail(feature, closest_point):
#     filtered_coords = []
#     start_adding = False  # Flag to start adding coordinates after the closest point
#     for coord in feature["geometry"]["coordinates"]:
#         if coord == closest_point:
#             start_adding = True  # Start adding coordinates after this point
#         if start_adding:
#             filtered_coords.append(coord)
#     return filtered_coords

In [5]:
# loads, processes, and saves the geojson file

# load geojson
with open(file_path, "r") as file:
    data = geojson.load(file)

# process geojson
filtered_features = []
for feature in data["features"]:
    if feature["geometry"]["type"] == "LineString":
        # find the closest point to the endpoint
        closest_point = find_closest_point(feature, endpoint)
        if closest_point:
            # filter to the closest point
            filtered_coords = filter_trail(feature, closest_point)
            if filtered_coords:
                # create new feature with the filtered geometry
                new_feature = geojson.Feature(
                    geometry=geojson.LineString(filtered_coords),
                    properties=feature["properties"],
                )
                filtered_features.append(new_feature)
    elif feature["geometry"]["type"] == "MultiLineString":
        # Handle MultiLineString
        filtered_multilines = []
        for line in feature["geometry"]["coordinates"]:
            # Treat each LineString in the MultiLineString
            temp_feature = {"geometry": {"coordinates": line}}
            closest_point = find_closest_point(temp_feature, endpoint)
            if closest_point:
                filtered_coords = filter_trail(temp_feature, closest_point)
                if filtered_coords:
                    filtered_multilines.append(filtered_coords)
        if filtered_multilines:
            new_feature = geojson.Feature(
                geometry=geojson.MultiLineString(filtered_multilines),
                properties=feature["properties"],
            )
            filtered_features.append(new_feature)

# save
filtered_geojson = geojson.FeatureCollection(filtered_features)
with open(file_path, "w") as file:
    geojson.dump(filtered_geojson, file, indent=2)

print("Filtered GeoJSON saved!")

Filtered GeoJSON saved!
